In [4]:
import pandas as pd
import numpy as np
import timeit

from dataclasses import dataclass, field, fields, asdict, is_dataclass
from typing import List, Dict, Optional, Any, Union, TypedDict, Tuple